## Compute Interaction Energies
* https://openmm.github.io/openmm-cookbook/latest/notebooks/cookbook/Computing%20Interaction%20Energies.html
* compute the interaction energy between two molecules


In [71]:
from openmm import *
from openmm.app import *
from openmm.unit import *

In [72]:
pdb = PDBFile('./results/merged_H.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)

In [73]:
for i in pdb.topology.chains():
    if i.index == 0:
        solvent = set([j.index for j in i.atoms() if j.residue.name in ('HOH', 'Cl', 'Na')])
        villin = set([j.index for j in i.atoms() if j.index not in solvent])
    else:
        ala = set([j.index for j in i.atoms()])

In [74]:
len(solvent)

8285

In [75]:
len(villin)

582

In [76]:
len(ala)

33

In [77]:
for force in system.getForces():
    if isinstance(force, NonbondedForce):
        force.setForceGroup(0)
        force.addGlobalParameter("villin_coulomb_scale", 1)
        force.addGlobalParameter("villin_lj_scale", 1)
        force.addGlobalParameter("ala_coulomb_scale", 1)
        force.addGlobalParameter("ala_lj_scale", 1)
        force.addGlobalParameter("solvent_coulomb_scale", 1)
        force.addGlobalParameter("solvent_lj_scale", 1)
        for i in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(i)
            force.setParticleParameters(i, 0, 0, 0)
            if i in villin:
                force.addParticleParameterOffset("villin_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("villin_lj_scale", i, 0, sigma, epsilon)
            elif i in ala:
                force.addParticleParameterOffset("ala_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("ala_lj_scale", i, 0, sigma, epsilon)
            elif i in solvent:
                force.addParticleParameterOffset("solvent_coulomb_scale", i, charge, 0, 0)
                force.addParticleParameterOffset("solvent_lj_scale", i, 0, sigma, epsilon)
        for i in range(force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = force.getExceptionParameters(i)
            force.setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        force.setForceGroup(2)

In [78]:
integrator = VerletIntegrator(0.001*picosecond)
context = Context(system, integrator)
context.setPositions(pdb.positions)

def energy(villin_coulomb_scale, villin_lj_scale, ala_coulomb_scale, ala_lj_scale, solvent_coulomb_scale, solvent_lj_scale):
    context.setParameter("villin_coulomb_scale", villin_coulomb_scale)
    context.setParameter("villin_lj_scale", villin_lj_scale)
    context.setParameter("ala_coulomb_scale", ala_coulomb_scale)
    context.setParameter("ala_lj_scale", ala_lj_scale)
    context.setParameter("solvent_coulomb_scale", solvent_coulomb_scale)
    context.setParameter("solvent_lj_scale", solvent_lj_scale)
    return context.getState(getEnergy=True, groups={0}).getPotentialEnergy()

total_coulomb = energy(1, 0, 1, 0, 1, 0)
villin_coulomb = energy(1, 0, 0, 0, 0, 0)
ala_coulomb = energy(0, 0, 1, 0, 0, 0)
sol_coulomb = energy(0, 0, 0, 0, 1, 0)
total_lj = energy(0, 1, 0, 1, 0, 1)
villin_lj = energy(0, 1, 0, 0, 0, 0)
ala_lj = energy(0, 0, 0, 1, 0, 0)
sol_lj = energy(0, 0, 0, 0, 0, 1)
print('Coulomb interaction energy:', total_coulomb - villin_coulomb - ala_coulomb-sol_coulomb)
print('LJ interaction energy:', total_lj - villin_lj - ala_lj - sol_lj)

Coulomb interaction energy: -6572.898544378346 kJ/mol
LJ interaction energy: 17088695530097.352 kJ/mol


In [79]:
for force in system.getForces():
    if isinstance(force, NonbondedForce):
        force.setForceGroup(0)
        force.addGlobalParameter("villin_scale", 1)
        force.addGlobalParameter("ala_scale", 1)
        force.addGlobalParameter("solvent_scale", 1)
        for i in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(i)
            force.setParticleParameters(i, 0, 0, 0)
            if i in villin:
                force.addParticleParameterOffset("villin_scale", i, charge, sigma, epsilon)
            elif i in ala:
                force.addParticleParameterOffset("ala_scale", i, charge, sigma, epsilon)
            elif i in solvent:
                force.addParticleParameterOffset("solvent_scale", i, charge, sigma, epsilon)
        for i in range(force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = force.getExceptionParameters(i)
            force.setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        force.setForceGroup(2)

In [80]:
integrator = VerletIntegrator(0.001*picosecond)
context = Context(system, integrator)
context.setPositions(pdb.positions)

def energy(villin_scale, ala_scale, solvent_scale):
    context.setParameter("villin_scale", villin_scale)
    context.setParameter("ala_scale", ala_scale)
    context.setParameter("solvent_scale", solvent_scale)
    return context.getState(getEnergy=True, groups={0}).getPotentialEnergy()

total = energy(1, 1, 1)
villin = energy(1, 0, 0)
ala = energy(0, 1, 0)
solvent = energy(0, 0, 1)

print('Total energy:', total)
print('Villin energy:', villin)
print('Ala energy:', ala)
print('Solvent energy:', solvent)
print('Interaction energy:', total - villin - ala - solvent)


Total energy: 17088695691278.877 kJ/mol
Villin energy: 17088695691278.877 kJ/mol
Ala energy: 17088695691278.877 kJ/mol
Solvent energy: 17088695691278.877 kJ/mol
Interaction energy: -34177391382557.754 kJ/mol
